1. 7000개 샘플링

In [2]:
import random
import glob
import shutil
import os

# 지정된 경로에서 .json 파일 리스트 가져오기
file_list = glob.glob('../data//train/train_*.json')

# 랜덤하게 7000개 파일 선택
selected_files = random.sample(file_list, 7000)

# 파일 이름 오름차순으로 정렬
selected_files.sort()

# 특정 디렉토리 생성 (이미 존재하는 경우에는 패스)
target_directory = "../data/selected_train/"
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

# 선택된 파일을 특정 디렉토리에 복사하고 이름 변경
for i, file in enumerate(selected_files):
    new_filename = f"{target_directory}train_{100000 + i}.json"
    shutil.copy(file, new_filename)

print("파일 복사 및 이름 변경이 완료되었습니다.")

파일 복사 및 이름 변경이 완료되었습니다.


2. 클래스별 샘플링

In [3]:
import json
from collections import Counter

# 라벨별 카운트를 저장할 Counter 객체 생성
label_counter = Counter()

# 선택된 파일에서 라벨 정보 읽기
for file in selected_files:
    with open(file, 'r') as f:
        data = json.load(f)
        shapes = data['shapes']
        for shape in shapes:
            label = shape['label']
            label_counter[label] += 1

# 라벨별 카운트 출력
print("라벨별 카운트:", label_counter)

라벨별 카운트: Counter({'01_ulcer': 4316, '05_bleeding': 2198, '04_lymph': 1353, '02_mass': 1124})


In [4]:
# 언더샘플링 로직 약 10초 소요
min_count = min(label_counter.values())
undersampled_files = []

for label, count in label_counter.items():
    candidate_files = [file for file in selected_files if any(shape['label'] == label for shape in json.load(open(file, 'r'))['shapes'])]
    
    # 가장 적은 라벨의 개수와 현재 라벨의 파일 개수 중 작은 것을 선택
    sample_count = min(min_count, len(candidate_files))
    
    # 에러 방지를 위해 sample_count가 0보다 클 경우에만 샘플링
    if sample_count > 0:
        undersampled_files.extend(random.sample(candidate_files, sample_count))

# 언더샘플링된 파일을 새로운 디렉토리에 복사
undersampled_directory = "../data/selected_train_UNDER/"
if not os.path.exists(undersampled_directory):
    os.makedirs(undersampled_directory)

for i, file in enumerate(undersampled_files):
    new_filename = f"{undersampled_directory}train_{100000 + i}.json"
    shutil.copy(file, new_filename)

print("언더샘플링 및 파일 복사가 완료되었습니다.")

언더샘플링 및 파일 복사가 완료되었습니다.
